## このカーネルについて

株式会社サポーターズ 主催**【オンライン勉強会】Kaggleタイタニックハンズオン**（学生向け）のKernelです。  
ハンズオンパートで使います。

0. モデル作成の準備
1. 単純なモデルを作成する
2. モデル作成を一緒に一回やって流れをつかむ
3. 性能が上がると考える選択肢を試す（サンプルコードのブロックを実行）

※勉強会自体は勉強会ページから申し込むと後追い視聴できます（ただし**学生限定**です）

### 変更履歴

| 開催時期 | Kernelのバージョン |
| ----|---- | 
| [2019/03](https://talent.supporterz.jp/events/2992699f-53ed-417b-abf7-7d7bba931037/) | [Version3](https://www.kaggle.com/ftnext/kaggle-spzcolab-online?scriptVersionId=11257494) |
| [2019/04](https://talent.supporterz.jp/events/d7384737-e2a6-4dc9-bc5e-f90e17e0924e/) | 最新のKernel |

## 基本操作

コードが書かれているブロックを **セル** と呼びます。  

1. セルをクリックして選択する（入力できる状態になる）
2. 選択したセルを実行する
    - 入力できる状態でShift+Enterキーを押す（こちらに慣れると簡単です）
    - 左側に表示される再生ボタン▶をクリック

## 0. モデル作成の準備

In [1]:
# 前処理に必要なモジュールの読み込み
# pandasは表形式のデータを扱うためのモジュール
import pandas as pd

### タイタニックコンペのフォルダ配置
```
├── ディレクトリ
│	└── 現在のKernel
└── input/
 	├── train.csv（モデル作成用データ）
 	├── test.csv（予測対象データ）
 	└── gender_submission.csv（提出練習用データ）
```
続くセルでデータを読み込む際は、**相対パス**でデータを指定しています

In [2]:
# 読み込んだデータはExcelの表のような形式で扱う（行と列がある）
# モデル作成用データの読み込み（生存か死亡か知っているデータ）
train_df = pd.read_csv('https://raw.githubusercontent.com/ftnext/spzcolab_titanic/master/input/train.csv')
# 予測対象データの読み込み（生存か死亡か知らないデータ）
test_df = pd.read_csv('https://raw.githubusercontent.com/ftnext/spzcolab_titanic/master/input/test.csv')

`read_csv`  
解説: https://note.nkmk.me/python-pandas-read-csv-tsv/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [3]:
# モデル作成用データのサイズを確認
# (行数, 列数) で表示される
train_df.shape

(891, 12)

In [4]:
# 予測対象データのサイズを確認
# モデル作成用データに対して1列少ない
test_df.shape

(418, 11)

`shape`  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.shape.html

In [5]:
# データの一部を見てみる
# モデル作成用データの上から5行を表示
# 参考: train_df.head(7) # 上から7行表示
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
# 予測対象データの上から5行を表示
# Survivedの列（生存か死亡かを表す）がないことが確認できる
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


`head`  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html

In [7]:
# 各列にどんな種類のデータが入っているか確かめる（数値なのか、文字列なのか）
# モデル作成用データの情報を確認（見方については後述）
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [8]:
# 予測対象データの情報を確認
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


`info`  
解説: https://note.nkmk.me/python-pandas-len-shape-size/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html

| infoによる情報 | 意味 |
| ---- | ---- | 
| int | 整数 |
| float | 浮動小数点数 |
| object | 文字列 |

| データの情報 | 列名 |　意味 |
| ----- | ----- | ----- |
| int | PassengerId | 乗客ID |
| int | **Pclass** | チケットの等級 (1, 2, 3) |
| int | SibSp | 同乗した兄弟姉妹/配偶者の人数 |
| int | Parch | 同乗した両親/子供の人数 |
| int | *Survived* | 0：死亡、1：生存 |
| float | **Age** | 年齢（推測があるため、浮動小数点数） |
| float | Fare | 運賃 |
| object | Name | 氏名 |
| object | **Sex** | 性別 |
| object | Ticket | チケット番号 |
| object | Cabin | 船室番号 |
| object | **Embarked** | 乗船した港の頭文字(S, Q, C) |

ref: https://www.kaggle.com/c/titanic/data

上の表で太字で示した列を使って、モデル作成を進めていきます。

- Pclass
- Age
- Sex
- Embarked

モデル作成に使う列を徐々に増やすというアプローチを取ります。

## 1. 単純なモデル

性別（Sex）を元に生死を予測するモデルを作成します。  
以下のルールで生死を予測することにします。

- 男性であれば、死亡
- 女性であれば、生存

注：この単純なモデルは機械学習で作るのではなく、人が決めたルールを実装して実現します

モデル作成用データについて、性別と生死を可視化すると、**男性の生存率は低く、女性の生存率は高い**ことが分かります。  
（ここで採用する予測ルールは全くの見当違いというわけではないということです）

![](https://raw.githubusercontent.com/ftnext/2019_slides/master/spz_online_titanic_handson/assets/survived_per_sex.png)

ここで用いた可視化手法は**ヒストグラム**と呼ばれます。  
男性、女性それぞれについて、Survivedのヒストグラムを描画しました。  

横軸0〜1の間に2本の柱があります。  
左側の柱はSurvivedの値が0〜0.5のデータの個数を表し、右側の柱はSurvivedの値が0.5〜1のデータの個数を表します。  
Survivedの取りうる値は0か1なので、**左側の柱は死亡者の人数、右側の柱は生存者の人数**を表します。

- 男性(male)のヒストグラムでは、死亡者 約450名、生存者 約100名と、**男性の生存率は低い**です
- 女性(female)のヒストグラムでは、死亡者 約80名、生存者 約230名と、**女性の生存率は高い**です

なお、可視化のコードはこちらのカーネルにあります：https://www.kaggle.com/ftnext/kaggle-spzcolab-201903

In [9]:
# モデル作成・性能評価に使うモジュールの読み込み
# scikit-learn（コードではsklearn）は、機械学習の様々なアルゴリズムや、モデルの評価ツールを提供するモジュール
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [10]:
# モデルが予測に使う情報（ここでは性別）をx, モデルが予測する情報（ここでは生死）をyとして取り出す
x = train_df['Sex']
y = train_df['Survived']

参考: 列の取り出し https://note.nkmk.me/python-pandas-index-row-column/  
ドキュメント: 10 minutes pandasの中のSelection https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html#selection

In [11]:
# xの値のうち、femaleが1に置き換わり、maleが0に置き換わる（コロンの左側に一致する要素が、コロンの右側の値に置き換わる）
# 注: Sexの取りうる値はfemaleかmale
# astype(int)でデータの型が文字列から整数へ変わることに対応
y_pred = x.map({'female': 1, 'male': 0}).astype(int)

`map`  
参考: https://note.nkmk.me/python-pandas-map-replace/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html

実行したコードのイメージ  

| `x` | `y_pred` |
| ---- | ---- |
| male | 0 |
| female | 1 |
|  :  | : |
| female | 1 |

`x`と対応するように`y_pred`を作成

タイタニックの場合、モデルの性能はaccuracyというスコアで評価される  
（**注**：性能を表すスコアは他にもある）

- 予測対象データ418名のうち生死を正しく予想できたものの割合
- 1に近いほど性能がよい

| ケース | 正解／誤り |
| ----- | ----- |
| 生存した乗客を生存と予測 | 正解 |
| 生存した乗客を死亡と予測 | 誤り |
| 死亡した乗客を生存と予測 | 誤り |
| 死亡した乗客を死亡と予測 | 正解 |

accuracy = (正解の総数) / (正解の総数 + 誤りの総数)

>Your score is the percentage of passengers you correctly predict. This is known simply as "accuracy”.

ref: https://www.kaggle.com/c/titanic#evaluation

In [12]:
# 予測y_predを実際の生死yで採点し、予測の正解率(accuracy)を表示
accuracy_score(y, y_pred)

0.7867564534231201

`accuracy_score`  
ドキュメント: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

In [13]:
# 予測対象データについて生死を予測する
# 予測対象データのSex列の取り出し
x_test = test_df['Sex']
# Sexの値を元に、生死を予測
y_test_pred = x_test.map({'female': 1, 'male': 0}).astype(int)

- 予測した結果から提出用データを作る
- 提出用データは以下の形式のCSVとするように指定されている（1が生存、0が死亡）

PassengerId | Survived
----- | -----
892 | 0
: | :
1309|1

- ref: https://www.kaggle.com/c/titanic#evaluation

In [14]:
# colabでの実行の際は省略
# # 提出用データの形式に変換
# submission = pd.DataFrame({
#     'PassengerId': test_df['PassengerId'],
#     'Survived': y_test_pred
# })
# # 提出用データ作成
# submission.to_csv('submission.csv', index=False)

提出は一緒に進めます  
（後追い視聴の方向けに手順をスライドにも記載します）

注：提出はブラウザの別のタブで行われますが、**このKernelのタブは閉じないでください**。  
（提出後、このタブに戻って、続きを進めます）

## 2. モデル作成を一緒に体験

単純なモデルよりも性能のいいモデル作りに挑戦

### SexとPclassからモデル作成

In [15]:
# SexとPclassから生死を予測するモデルを作ることにする
columns = ['Pclass', 'Sex']

Pclass=1,2,3それぞれについて、Survivedのヒストグラムを描画

![](https://raw.githubusercontent.com/ftnext/2019_slides/master/spz_online_titanic_handson/assets/survived_per_pclass.png)

Pclassの値によって生存率が異なるため、Pclassは生死の予測に使えると考えられます

- Pclass=1は生存者が過半数を超えている
- Pclass=2は生存者が半分程度
- Pclass=3は生存者が少ない（4分の1程度）

In [16]:
# モデル作成に使う情報をX, モデルが予測する情報（ここでは生死）をyとして取り出す（Xとyという変数名が多い）
X = train_df[columns].copy()
y = train_df['Survived']
# 予測対象データについて、予測に使う情報を取り出しておく
X_test = test_df[columns].copy()

In [17]:
# モデル作成に使うデータを確認
X.head()

,Pclass,Sex
0,3,male
1,1,female
2,3,female
3,1,female
4,3,male


- いくつかの文字列の値を取る変数：カテゴリ変数
    - Sex: male, female
- 文字列を整数に変換する
    - Sex: male=1, female=0として置き換え
    
モデル作成に使うデータ`X`について変換する際は、モデルが予測するデータ`X_test`についても**同様の変換をする**ことがポイントとなります。

In [18]:
# 性別（female/male）を0/1に変換する（maleとfemaleのままではモデル作成時に扱えない）
# カテゴリを整数に置き換えるための「辞書」を用意
gender_map = {'female': 0, 'male': 1}
# 引数の辞書でコロンの左側（キー）に一致する要素が、コロンの右側の値に置き換わる（femaleが0に置き換わり、maleが1に置き換わる）
X['Gender'] = X['Sex'].map(gender_map).astype(int)
X_test['Gender'] = X_test['Sex'].map(gender_map).astype(int)

Sexを0/1に変換したあとのX（イメージ）  
**Genderという列が追加される**

| Pclass | Sex | Gender |
| ---- | ---- | ---- |
| 2 | male | 1 |
| 3 | female | 0 |
| : | : | : |

In [19]:
# Sexに代えてGenderを使うため、Sex列を削除する
X = X.drop(['Sex'], axis=1)
X_test = X_test.drop(['Sex'], axis=1)

`drop`  
参考: https://note.nkmk.me/python-pandas-drop/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

In [20]:
# モデル作成に使うデータについて、前処理した後の状態を確認
X.head()

,Pclass,Gender
0,3,1
1,1,0
2,3,0
3,1,0
4,3,1


このあとモデルを作成します。  
モデルを作成したらすぐに予測対象データについて予測するのではなく、**どの程度の性能のモデルなのか**確認します。  
モデルの性能確認のために、モデル作成に使うデータをランダムに2つに分けます（`train_test_split`を使います）

- モデル作成に使うデータのうち、例えば7割でモデルを作る
- 残りの3割でモデルの性能を評価する
    - モデルの性能が問題なければ、予測対象データを予測する

![](https://raw.githubusercontent.com/ftnext/2019_slides/master/spz_Jan_titanic_handson/assets/201901kaggel_talk.010.png)

気になる方へ：  
ランダムに分けていますが、`random_state`引数の値が同じなら、**何回実行しても同じ**ようにランダムに分かれます

In [21]:
# 今回のハンズオンは7:3に分けて進める
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)

`train_test_split`  
参考: https://docs.pyq.jp/python/machine_learning/tips/train_test_split.html  
ドキュメント: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

| 変数名 | 用途 |
| ----- | ----- |
| X_train | 学習用データ（乗客の属性） |
| y_train | 学習用データ（生死） |
| X_val | 性能評価用データ（乗客の属性） |
| y_val | 性能評価用のデータ（生死） |

※学習とは、データとアルゴリズムからモデル（＝推論のルール）を作成すること

In [22]:
# 学習用データの数の確認
len(y_train)

623

In [23]:
# 性能評価用のデータの数の確認
len(y_val)

268

In [24]:
# 決定木というアルゴリズムを使ったモデルを用意
model = DecisionTreeClassifier(random_state=1, criterion='entropy', max_depth=3, min_samples_leaf=2)
# モデル作成は以下の1行（ここまでの前処理に対してたった1行！）で完了する
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

参考: https://scikit-learn.org/stable/modules/tree.html#classification  
ドキュメント: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [25]:
# 性能評価用データについて生死を予測
pred = model.predict(X_val)
# 性能確認:accuracyを算出して表示
accuracy_score(y_val, pred)

0.746268656716418

参考情報  
別のモデルを作ったときに、以前に作ったモデルよりスコアが低ければ、提出しないという判断ができます  
（Kaggleに提出しなくても試行錯誤が進められる。提出回数には日次の上限がある）

In [26]:
# X_testについて生死を予測（予測対象データからSexとPclassをX_testとして取り出し、Xと同様の前処理を行っている）
pred = model.predict(X_test)

In [27]:
# colabでの実行の際は省略
# # 提出用データの形式に変換
# submission = pd.DataFrame({
#     'PassengerId': test_df['PassengerId'],
#     'Survived': pred
# })
# # 提出用データ作成
# submission.to_csv('submission.csv', index=False)

## 3. 用意した選択肢を試す

欠損値の対応を新しく扱う

### （選択肢1）Ageを追加

In [28]:
# モデルの予測に使う情報にAgeを追加
columns = ['Pclass', 'Sex', 'Age']

Ageについてヒストグラムを作成（8歳で一つの柱としている）  
1つの柱のうち、生存者と死亡者を色分けして表示している  

![](https://raw.githubusercontent.com/ftnext/2019_slides/master/spz_online_titanic_handson/assets/age_hist_stacked_survived.png)

Ageの層によって生存率が異なるため、Ageは生死の予測に使えると考えられます。

助かっている割合が高い年齢層
- 0歳〜8歳（半数以上）
- 10代前半、30代後半、50〜60歳（半数程度）

In [29]:
# モデル作成に使う情報をX, モデルが予測する情報（ここでは生死）をyとして取り出す
X = train_df[columns].copy()
y = train_df['Survived']
# 予測対象データについて、予測に使う情報を取り出しておく
X_test = test_df[columns].copy()

In [30]:
# モデル作成に使うデータを確認
X.head()

,Pclass,Sex,Age
0,3,male,22.0
1,1,female,38.0
2,3,female,26.0
3,1,female,35.0
4,3,male,35.0


**欠損値**について

- 欠けたデータのこと（missing value、欠測値とも呼ばれる）
    - pandasで扱う際は、NaNと表示される（`train_df.head()`のCabin列）
    - `train_df.info()`の結果に714や204など、891よりも小さい値があった
- 欠損の要因：データの収集過程で抜けてしまったなど
- 機械学習のツールは **一般に欠損値に対処できない**（そのためデータを前処理する）

In [31]:
# モデル作成に使うデータの欠損値の確認
X.isnull().sum()

Pclass      0
Sex         0
Age       177
dtype: int64

In [32]:
# モデルが予測するデータの欠損値の確認
X_test.isnull().sum()

Pclass     0
Sex        0
Age       86
dtype: int64

`isnull().sum()`  
参考: https://note.nkmk.me/python-pandas-nan-judge-count/

- Ageがモデル作成用・性能評価用とで20%程度欠けていることがわかった
- 削除すると貴重なデータが減るので、**埋める**
- モデル作成用データの平均値（=年齢の総和/個数）で埋める

In [33]:
# Ageの平均値の算出
age_mean = X['Age'].mean()
print(f'Age mean: {age_mean}')

Age mean: 29.69911764705882


`mean`  
参考: https://deepage.net/features/pandas-mean.html  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.mean.html

In [34]:
# 平均値を小数第2位で四捨五入して使う(round関数)
# Ageの欠損を平均値で埋めた列AgeFillを追加
X['AgeFill'] = X['Age'].fillna(round(age_mean, 1))
X_test['AgeFill'] = X_test['Age'].fillna(round(age_mean, 1))

`fillna`  
参考: https://note.nkmk.me/python-pandas-nan-dropna-fillna/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html

Ageの欠損を埋めたあとのX（イメージ）  
**AgeFillという列が追加される**

| Pclass | Sex | Age | AgeFill |
| ---- | ---- | ---- | ---- |
| 2 | male | NaN | 29.7 |
| 3 | female | 31 | 31 |
| : | : | : | : |

In [35]:
# 欠損を含むAge列を削除（年齢の情報はAgeFill列を参照する）
X = X.drop(['Age'], axis=1)
X_test = X_test.drop(['Age'], axis=1)

`drop`  
参考: https://note.nkmk.me/python-pandas-drop/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html

In [36]:
# モデル作成に使うデータの欠損値の確認
X.isnull().sum()

Pclass     0
Sex        0
AgeFill    0
dtype: int64

In [37]:
# モデルが予測するデータの欠損値の確認
X_test.isnull().sum()

Pclass     0
Sex        0
AgeFill    0
dtype: int64

In [38]:
# 性別（female/male）を0/1に変換する（「2.モデル作成を一緒に体験」と同様）
gender_map = {'female': 0, 'male': 1}
X['Gender'] = X['Sex'].map(gender_map).astype(int)
X_test['Gender'] = X_test['Sex'].map(gender_map).astype(int)

In [39]:
# Sexに代えてGenderを使うため、Sex列を削除する
X = X.drop(['Sex'], axis=1)
X_test = X_test.drop(['Sex'], axis=1)

In [40]:
# モデル作成に使うデータについて、前処理した後の状態を確認
X.head()

,Pclass,AgeFill,Gender
0,3,22.0,1
1,1,38.0,0
2,3,26.0,0
3,1,35.0,0
4,3,35.0,1


In [41]:
# 今回のハンズオンではモデル作成用データを7:3に分けて進める
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)

In [42]:
# 学習用データの数の確認
len(y_train)

623

In [43]:
# 性能評価用のデータの数の確認
len(y_val)

268

In [44]:
# 決定木というアルゴリズムを使ったモデルを用意
model = DecisionTreeClassifier(random_state=1, criterion='entropy', max_depth=3, min_samples_leaf=2)
# モデル作成は以下の1行（ここまでの前処理に対してたった1行！）で完了する
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [45]:
# 性能評価用データについて生死を予測
pred = model.predict(X_val)
# 性能確認：accuracyを算出して表示
accuracy_score(y_val, pred)

0.7686567164179104

In [46]:
# 予測対象データについて生死を予測
pred = model.predict(X_test)

In [47]:
# colabでの実行の際は省略
# # 提出用データの形式に変換
# submission = pd.DataFrame({
#     'PassengerId': test_df['PassengerId'],
#     'Survived': pred
# })
# # 提出用データ作成
# submission.to_csv('submission.csv', index=False)

### （選択肢2）Embarkedを追加

In [48]:
# モデルの予測に使う情報にEmbarkedを追加
columns = ['Pclass', 'Sex', 'Embarked']

Embarked=S,Q,Cそれぞれについて、Survivedのヒストグラムを描画

![](https://raw.githubusercontent.com/ftnext/2019_slides/master/spz_online_titanic_handson/assets/survived_per_embarked.png)

港によって生存率が異なるため、Embarkedを生死の予測に加えてみます

- 港Cは生存者が半分程度
- 港S, Qは生存者が3分の1程度

In [49]:
# モデル作成に使う情報をX, モデルが予測する情報（ここでは生死）をyとして取り出す
X = train_df[columns].copy()
y = train_df['Survived']
# 予測対象データについて、予測に使う情報を取り出しておく
X_test = test_df[columns].copy()

In [50]:
# モデル作成に使うデータを確認
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [51]:
# モデル作成に使うデータの欠損値の確認
X.isnull().sum()

Pclass      0
Sex         0
Embarked    2
dtype: int64

In [52]:
# モデルが予測するデータの欠損値の確認
X_test.isnull().sum()

Pclass      0
Sex         0
Embarked    0
dtype: int64

Embarkedがモデル作成用で数件欠けている

*※欠損値の対応の詳細は選択肢1を確認してください*

モデル作成用データで一番多く登場する値（一番多くの人が乗っている港）で埋める

In [53]:
# 一番多くの人が乗っている港の取得
embarked_freq = X['Embarked'].mode()[0]
print(f'Embarked freq: {embarked_freq}')

Embarked freq: S


`mode`  
参考: https://note.nkmk.me/python-pandas-mode/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.mode.html

In [54]:
# Embarkedの欠損を平均値で埋めた列EmbarkedFillを追加
X['EmbarkedFill'] = X['Embarked'].fillna(embarked_freq)
X_test['EmbarkedFill'] = X_test['Embarked'].fillna(embarked_freq)

Embarkedの欠損を埋めたあとのX（イメージ）  
**EmbarkedFillという列が追加される**

| Pclass | Sex | Embarked | EmbarkedFill |
| ---- | ---- | ---- | ---- |
| 2 | male | NaN | S |
| 3 | female | C | C |
| 1 | female | Q | Q |
| : | : | : | : |

In [55]:
# 欠損を含むEmbarked列を削除（乗船した港の情報はEmbarkedFill列を参照する）
X = X.drop(['Embarked'], axis=1)
X_test = X_test.drop(['Embarked'], axis=1)

In [56]:
# モデル作成に使うデータの欠損値の確認
X.isnull().sum()

Pclass          0
Sex             0
EmbarkedFill    0
dtype: int64

In [57]:
# 性別（female/male）を0/1に変換する（「2.モデル作成を一緒に体験」と同様）
gender_map = {'female': 0, 'male': 1}
X['Gender'] = X['Sex'].map(gender_map).astype(int)
X_test['Gender'] = X_test['Sex'].map(gender_map).astype(int)

In [58]:
# Sexに代えてGenderを使うため、Sex列を削除する
X = X.drop(['Sex'], axis=1)
X_test = X_test.drop(['Sex'], axis=1)

カテゴリ変数EmbarkedFillはダミー変数化する

- S=1, Q=2, C=3と整数に置き換える
    - **本来なかった大小関係が想定されてしまう**
- (S, Q, C)という形式で整数に置き換える

| EmbarkedFillの値 | 置き換えたあと |
| ----- | ----- |
| S | (1, 0, 0) |
| Q | (0, 1, 0) |
| C | (0, 0, 1) |

In [59]:
# EmbarkedFill（S, Q, Cという3カテゴリ）をダミー変数にする
# （EmbarkedFill列が消え、EmbarkedFill_S, EmbarkedFill_Q, EmbarkedFill_C列が追加される）
X = pd.get_dummies(X, columns=['EmbarkedFill'])
X_test = pd.get_dummies(X_test, columns=['EmbarkedFill'])

`get_dummies`  
参考: https://note.nkmk.me/python-pandas-get-dummies/  
ドキュメント: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

In [60]:
# モデル作成に使うデータについて、前処理した後の状態を確認
X.head()

,Pclass,Gender,EmbarkedFill_C,EmbarkedFill_Q,EmbarkedFill_S
0,3,1,0,0,1
1,1,0,1,0,0
2,3,0,0,0,1
3,1,0,0,0,1
4,3,1,0,0,1


In [61]:
# 今回のハンズオンではモデル作成用データを7:3に分けて進める
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)

In [62]:
# 学習用データの数の確認
len(y_train)

623

In [63]:
# 性能評価用のデータの数の確認
len(y_val)

268

In [64]:
# 決定木というアルゴリズムを使ったモデルを用意
model = DecisionTreeClassifier(random_state=1, criterion='entropy', max_depth=3, min_samples_leaf=2)
# モデル作成は以下の1行（ここまでの前処理に対してたった1行！）で完了する
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [65]:
# 性能評価用データについて生死を予測
pred = model.predict(X_val)
# 性能確認：accuracyを算出して表示
accuracy_score(y_val, pred)

0.7611940298507462

In [66]:
# 予測対象データについて生死を予測
pred = model.predict(X_test)

In [67]:
# colabでの実行の際は省略
# # 提出用データの形式に変換
# submission = pd.DataFrame({
#     'PassengerId': test_df['PassengerId'],
#     'Survived': pred
# })
# # 提出用データ作成
# submission.to_csv('submission.csv', index=False)

### 練習問題: AgeとEmbarkedの両方を追加

In [68]:
# train_df, test_dfからSex, Pclass, Age, Embarkedを取り出す

In [69]:
# Ageの欠損への対応

In [70]:
# Embarkedの欠損への対応

In [71]:
# がテゴリ変数Sexへの対応

In [72]:
# カテゴリ変数Embarkedへの対応

In [73]:
# モデル作成

In [74]:
# 性能確認

In [75]:
# 提出用データ作成

## おまけ 3.（選択肢1）の補足

年齢を**年代**として扱う

In [76]:
# モデルの予測に使う情報にAgeを追加
columns = ['Pclass', 'Sex', 'Age']

ヒストグラムのようにAgeをおよそ8歳刻みの年代に分けるアプローチを紹介。  
年齢が増えるほど傾向が変わる（例：29歳より28歳が助かりやすく、28歳よりも27歳が助かりやすい）とは考えにくい。  
しかし、年代によって傾向が変わることは十分考えられる。（例：20代よりも10代のほうが助かりやすい）

![](https://raw.githubusercontent.com/ftnext/2019_slides/master/spz_online_titanic_handson/assets/age_hist_stacked_survived.png)

In [77]:
# モデル作成に使う情報をX, モデルが予測する情報（ここでは生死）をyとして取り出す
X = train_df[columns].copy()
y = train_df['Survived']
# 予測対象データについて、予測に使う情報を取り出しておく
X_test = test_df[columns].copy()

In [78]:
# モデル作成に使うデータを確認
X.head()

,Pclass,Sex,Age
0,3,male,22.0
1,1,female,38.0
2,3,female,26.0
3,1,female,35.0
4,3,male,35.0


In [79]:
# モデル作成に使うデータの欠損値の確認
X.isnull().sum()

Pclass      0
Sex         0
Age       177
dtype: int64

In [80]:
# モデルが予測するデータの欠損値の確認
X_test.isnull().sum()

Pclass     0
Sex        0
Age       86
dtype: int64

In [81]:
# Ageの平均値の算出
age_mean = X['Age'].mean()
print(f'Age mean: {age_mean}')

Age mean: 29.69911764705882


In [82]:
# 平均値を小数第2位で四捨五入して使う(round関数)
# Ageの欠損を平均値で埋めた列AgeFillを追加
X['AgeFill'] = X['Age'].fillna(round(age_mean, 1))
X_test['AgeFill'] = X_test['Age'].fillna(round(age_mean, 1))

In [83]:
# 欠損を含むAge列を削除（年齢の情報はAgeFill列を参照する）
X = X.drop(['Age'], axis=1)
X_test = X_test.drop(['Age'], axis=1)

In [84]:
# モデル作成に使うデータの欠損値の確認
X.isnull().sum()

Pclass     0
Sex        0
AgeFill    0
dtype: int64

In [85]:
# モデルが予測するデータの欠損値の確認
X_test.isnull().sum()

Pclass     0
Sex        0
AgeFill    0
dtype: int64

In [86]:
# AgeFillを最小と最大の間で10分割
age_band = pd.cut(X['AgeFill'], 10)
# 区間に含まれる年代の順に表示
age_band.value_counts()

(24.294, 32.252]    346
(16.336, 24.294]    177
(32.252, 40.21]     118
(40.21, 48.168]      70
(0.34, 8.378]        54
(8.378, 16.336]      46
(48.168, 56.126]     45
(56.126, 64.084]     24
(64.084, 72.042]      9
(72.042, 80.0]        2
Name: AgeFill, dtype: int64

`cut`および`value_counts`  
参考: https://note.nkmk.me/python-pandas-cut-qcut-binning/

In [87]:
# 年代を若い順に0,1,2,...,9と設定
for df in [X, X_test]:
    df.loc[df['AgeFill'] <= 8.378, 'AgeFill'] = 0
    df.loc[(df['AgeFill'] > 8.378) & (df['AgeFill'] <= 16.336), 'AgeFill'] = 1
    df.loc[(df['AgeFill'] > 16.336) & (df['AgeFill'] <= 24.294), 'AgeFill'] = 2
    df.loc[(df['AgeFill'] > 24.294) & (df['AgeFill'] <= 32.252), 'AgeFill'] = 3
    df.loc[(df['AgeFill'] > 32.252) & (df['AgeFill'] <= 40.21), 'AgeFill'] = 4
    df.loc[(df['AgeFill'] > 40.21) & (df['AgeFill'] <= 48.168), 'AgeFill'] = 5
    df.loc[(df['AgeFill'] > 48.168) & (df['AgeFill'] <= 56.126), 'AgeFill'] = 6
    df.loc[(df['AgeFill'] > 56.126) & (df['AgeFill'] <= 64.084), 'AgeFill'] = 7
    df.loc[(df['AgeFill'] > 64.084) & (df['AgeFill'] <= 72.042), 'AgeFill'] = 8
    df.loc[df['AgeFill'] > 72.042, 'AgeFill'] = 9
    df['AgeFill'] = df['AgeFill'].astype(int) # floatからintに変更

`loc`  
例: `df.loc[(df['AgeFill'] > 8.378) & (df['AgeFill'] <= 16.336), 'AgeFill']`  
→AgeFillが8.378以上、かつ、16.336未満のすべての行のAgeFillの値（コードでは、右辺の値1に変更している）  
参考: https://note.nkmk.me/python-pandas-at-iat-loc-iloc/

In [88]:
# 性別（female/male）を0/1に変換する（「2.モデル作成を一緒に体験」と同様）
gender_map = {'female': 0, 'male': 1}
X['Gender'] = X['Sex'].map(gender_map).astype(int)
X_test['Gender'] = X_test['Sex'].map(gender_map).astype(int)

In [89]:
# Sexに代えてGenderを使うため、Sex列を削除する
X = X.drop(['Sex'], axis=1)
X_test = X_test.drop(['Sex'], axis=1)

In [90]:
# モデル作成に使うデータについて、前処理した後の状態を確認
X.head()

,Pclass,AgeFill,Gender
0,3,2,1
1,1,4,0
2,3,3,0
3,1,4,0
4,3,4,1


In [91]:
# 今回のハンズオンではモデル作成用データを7:3に分けて進める
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=1)

In [92]:
# 学習用データの数の確認
len(y_train)

623

In [93]:
# 性能評価用のデータの数の確認
len(y_val)

268

In [94]:
# 決定木というアルゴリズムを使ったモデルを用意
model = DecisionTreeClassifier(random_state=1, criterion='entropy', max_depth=3, min_samples_leaf=2)
# モデル作成は以下の1行（ここまでの前処理に対してたった1行！）で完了する
model.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=1,
            splitter='best')

In [95]:
# 性能評価用データについて生死を予測
pred = model.predict(X_val)
# 性能確認：accuracyを算出して表示
accuracy_score(y_val, pred)

0.7649253731343284

In [96]:
# 予測対象データについて生死を予測
pred = model.predict(X_test)

In [97]:
# colabでの実行の際は省略
# # 提出用データの形式に変換
# submission = pd.DataFrame({
#     'PassengerId': test_df['PassengerId'],
#     'Survived': pred
# })
# # 提出用データ作成
# submission.to_csv('submission.csv', index=False)

### 変更履歴
https://www.kaggle.com/ftnext/kaggle-spzcolab-201901 をベースにやることを絞ってアップデートした  
ref: https://github.com/PyDataTokyo/pydata-tokyo-tutorial-1

V3にて試したいコード以外でsubmission.csvが作られないようにコメントアウトした。
（V2までは単純なモデルでコミットしても、全てのコードが実行されるために、3（選択肢2）のモデルの予測結果が作成されるという不具合があった）